In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl

from scipy.stats import mode
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

## **СТАТИСТИКА**

**Содержание**

[Описание задачи](#descr)

 1.[ Загрузка данных](#loading_data)
 
 2.[ Статистика](#stats)
   * [ Диаграмма корреляции](#сorrel_chart)
   * [ HP](#HP)
   * [ Attack и Defence](#attack_defence)
   * [ Гистограмма количество покемонов в группах](#histogram)
   * [ Специальные возможности (Sp_attack и Sp_defence)](#sp_features)
   * [ Speed](#speed)

   
3.[ Вывод](#conclusion)

## ***Описание задачи*** <a class='heading' id='descr'>
**Name**: Имя каждого покемона

**Total**: сумма всех статистических данных, которые приходят после этого, общее руководство о том, насколько силен покемон

**HP**: хит-пойнты, или здоровье, определяют, сколько урона покемон может выдержать, прежде чем упасть в обморок

**Attack**: базовый модификатор для обычных атак (например. Царапина, Удар)

**Defence**: сопротивление базовому урону от обычных атак (защита)

**SP_Atk**: специальная атака, базовый модификатор для специальных атак (например, огненный взрыв, пузырьковый луч)

**SP_Def**: устойчивость к базовому урону от специальных атак

**Speed**: скорость определяет, какой покемон атакует первым в каждом раунде

## ***Загрузка данных*** <a class='heading' id='loading_data'>

In [ ]:
DATASET = '../input/pokemons-and-there-stats/pokemon.csv'
PREPARED_DATASET = './prepared_pokemon.csv'

In [ ]:
df = pd.read_csv(DATASET, sep=',')
df.head(10)

In [ ]:
df.tail(5)

In [ ]:
df.info()

## ***Статистика*** <a class='heading' id='stats'>

In [ ]:
round(df.describe(), 2)

In [ ]:
#Мода
print('Total', df.Total.mode(), sep='\n'),
print('HP', df.HP.mode(), sep='\n'),
print('Attack', df.Attack.mode(), sep='\n'),
print('Defence', df.Defence.mode(), sep='\n'),
print('Sp_attack', df.Sp_attack.mode(), sep='\n'),
print('Sp_defence', df.Sp_defence.mode(), sep='\n'),
print('Speed', df.Speed.mode(), sep='\n')


In [ ]:
#Медиана -> Quantile = 50%
print('Total: ', df.Total.median()),
print('HP: ', df.HP.median()),
print('Attack: ', df.Attack.median()),
print('Defence: ', df.Defence.median()),
print('Sp_attack: ', df.Sp_attack.median()),
print('Sp_defence: ', df.Sp_defence.median()),
print('Speed: ', df.Speed.median())


### ***Диаграмма корреляции*** <a class='subtitle' id='сorrel_chart'>

In [ ]:
plt.figure(figsize=(12,10), dpi=80)
sns.heatmap(df.corr(), xticklabels=df.corr().columns, yticklabels=df.corr().columns, cmap='RdYlGn', center=0, annot=True)

plt.title('Correlogram of pokemon', fontsize=22)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.show()

### ***HP*** <a class='subtitle' id='HP'>   

In [ ]:
plt.figure(figsize=(7,5), dpi=80)
sns.kdeplot(df['HP'], shade=True, color='dodgerblue', alpha=.2)

plt.axvline(x = np.quantile(df.HP, q=0.25), color='red', linestyle='-.', label='Quantile = 25%') # Quantile=50
plt.axvline(x = np.quantile(df.HP, q=0.75), color='red', linestyle='--', label='Quantile = 75%') # Quantile=82
plt.axvline(x = df['HP'].median(), color='b', label='Median') # Median = 68

plt.legend()
plt.show()
#По оси У представлена плотность распределения (или вероятность появления того или иного поведения выборки)

 Разобьем покемонов на 3 группы по столбцу HP (Здоровье):

1. Слабая (**Weak**) - покемоны, которые обладают слабым здоровьем (от 0 до 25%);
2. Средняя (**Average**) - покемоны, которые обладают средним здоровьем (от 25% до 75%);
3. Сильная (**Strong**) - покемоны, которые обладают сильным здоровьем (от 75% и выше).

In [ ]:
Group_HP = ""
df['Group_HP'] = Group_HP
df['Group_HP'] = np.where(df.HP <= np.quantile(df.HP, q=0.25), 'Weak',
                         (np.where(df.HP<=np.quantile(df.HP, q=0.75), 'Average', 'Strong')))
df

### ***Attack и Defence*** <a class='subtitle' id='attack_defence'>
 Тоже самое сделано и c Attack и Defence.
  


In [ ]:
# Функция для показа квантилей и медиан на графике
def show_line(n, a):     # n - входящий столбец, к примеру df.HP
    z = a.axvline(x = np.quantile(n, q=0.25), color='r', linestyle='-.', label='Quantile = 25%')
    b = a.axvline(x = np.quantile(n, q=0.75), color='r', linestyle='--', label='Quantile = 75%')
    c = a.axvline(x = n.median(), color='b', label='Median')
    return z, b, c

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(5,3), dpi=100)
ax1, ax2 = ax.flatten()
fig.set_size_inches(9, 4)
fig.subplots_adjust(wspace=0.3, hspace=0.3)
#Attack
sns.kdeplot(df['Attack'], ax=ax1, shade=True, color='magenta', alpha=.2)
show_line(df.Attack, ax1)
ax1.legend()
#Defence
sns.kdeplot(df['Defence'], ax=ax2, shade=True, color='springgreen', alpha =0.2)
show_line(df.Defence, ax2)
ax2.legend()

plt.show()

  Разобьем покемонов на 3 группы по столбцу Attack (Атака):

1. Слабая (**Weak**) - покемоны, которые обладают слабым ударом (от 0 до 55);
2. Средняя (**Average**) - покемоны, которые обладают средним ударом (от 56 до 100);
3. Сильная (**Strong**) - покемоны, которые обладают сильным ударом (от 101 и выше).

И разобьем также покемонов на 3 группы по столбцу Defence (Защита):

1. Слабая (**Weak**) - покемоны, которые обладают слабой защитой (от 0 до 50);
2. Средняя (**Average**) - покемоны, которые обладают средней защитой (от 51 до 90);
3. Сильная (**Strong**) - покемоны, которые обладают сильной защитой (от 91 и выше).

In [ ]:
#Функция добавления
def add_col(x):          #x - значение принимаемого столбца, к примеру df['Attack']
    a = np.where(x <= np.quantile(x, q=0.25), 'Weak',
                         (np.where(x<=np.quantile(x, q=0.75), 'Average', 'Strong')))
    return a   

In [ ]:
#Добавляю новые колонки
df['Group_Attack'] = add_col(df['Attack'])
df['Group_Defence'] = add_col(df['Defence'])

In [ ]:
#Формирую удобный столбец для изучение данных
df['Group'] = df['Group_HP'].str[0] + df['Group_Attack'].str[0] + df['Group_Defence'].str[0]
df

### ***Гистограмма количество покемонов в группах*** <a class='subtitle' id='histogram'>

In [ ]:
count_grp = df.groupby('Group')['Name'].count()
count_grp

In [ ]:
def autolabel(rects, labels=None, height_factor=1.01): # Функция для выводы значения на гистограмме
    for i, rect in enumerate(rects):
        height = rect.get_height()
        if labels is not None:
            try:
                label = labels[i]
            except (TypeError, KeyError):
                label = ' '
        else:
            label = '%d' % int(height)
        ax.text(rect.get_x() + rect.get_width()/2., height_factor*height,
                '{}'.format(label),
                ha='center', va='bottom')

plt.figure(figsize=(10, 10), dpi=80)
color_rectangle = np.random.rand(7, 3)
ax = count_grp.plot(kind='bar', fontsize=12, color = color_rectangle)
ax.set_title('The number of Pokemon in the group')
autolabel(ax.patches, count_grp, height_factor=1.01)

### ***Специальные возможности (Sp_attack и Sp_defence)*** <a class='subtitle' id='sp_features'>

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(4,2), dpi=100)
ax1, ax2 = ax.flatten()
fig.set_size_inches(9, 4)
fig.subplots_adjust(wspace=0.3, hspace=0.3)
#Sp_attack
sns.kdeplot(df.Sp_attack,ax=ax1, shade=True, color='g', alpha=.15)
show_line(df.Sp_attack, ax1)
ax1.legend()
#Sp_defence
sns.kdeplot(df.Sp_defence,ax=ax2, shade=True, color='gold', alpha=.3)
show_line(df.Sp_defence, ax2)
ax2.legend()

plt.show()

In [ ]:
df['Group_sp_a'] = add_col(df['Sp_attack'])
df['Group_sp_d'] = add_col(df['Sp_defence'])
df

### ***Speed*** <a class='subtitle' id='speed'>

In [ ]:
plt.figure(figsize=(7,5), dpi=80)
sns.kdeplot(df['Speed'], shade=True, color='darkgray', alpha=.3)
show_line(df.Speed, plt)

plt.legend()
plt.show()

И разобьем также покемонов на 3 группы по столбцу Speed (Скорость):

1. Медленный (**Slow**) - покемоны, которые имеют медленную скорость в атаке (от 0 до 25%);
2. Средняя (**Average**) - покемоны, которые имеют среднюю скорость в атаке (от 25% до 75%);
3. Быстрые (**Fast**) - покемоны, которые имеют быструю скорость в атаке (от 75% и выше).

In [ ]:
Group_Speed = ""
df['Group_Speed'] = Group_Speed
df['Group_Speed'] = np.where(df.Speed <= np.quantile(df.Speed, q=0.25), 'Slow',
                         (np.where(df.Speed<=np.quantile(df.Speed, q=0.75), 'Average', 'Fast')))
df

## ***Вывод*** <a class='heading' id='conclusion'>

Отметим покемонов из слабой группы (WWW) и из сильной группы (SSS). Другие группы с покемонами, как и самих покемонов, нужно рассматривать индивидуально под определенную задачу.

*Самая сильная группа - SSS (37 покемонов)*

**Mega Eternatus** - сильнейший по здоровью, по защите и по спец.защите. Но в атаке, скорости и спец.атаке, его превосходят:

   **Mega Mewtwo** - самая сильная атака (но защита и здоровье на ближе к среднему уровню), cпец.атака мощная.
   **Mega Mewtwo X** - самая сильная спец.атака. в остальном схож с Mega Mewtwo.
   **Regieleki** - быстрее всех атакует.
Также **Blissey** имеет равносильное здоровье как и Mega Eternatus.
   
*Самые слабые покемоны представлены в группе WWW (103 покемона)*

   **Shedinja** имеет самое слабое здоровье, но атака у покемона на среднем уровне.
   У покемонов **Happiny** и **Chansey** слабые атаки и слабая защита, но оба покемона имеют высокий уровень здоровья (особенно Chansey). Также Chansey имеет очень хорошую спец.защиту.

In [ ]:
#Сильный покемон из предоставленной выборки.
df[df.Name == 'Mega Eternatus']

In [ ]:
#Покемоны с сильным здоровьем
df[df.HP == 255]

In [ ]:
#Покемон с сильной атакой
df[df.Attack == 190]

In [ ]:
#Покемон с сильной защитой
df[df.Defence == 250]

In [ ]:
#Покемон с сильной спец.атакой
df[df.Sp_attack == 194]

In [ ]:
#Покемон с сильной спец.защитой
df[df.Sp_defence== 250]

In [ ]:
#Покемон быстрый в атаке
df[df.Speed== 200]

In [ ]:
#Покемон со слабым здоровьем
df[df.HP == 1]

In [ ]:
#Покемоны со слабой атакой и защитой 
df[df.Attack == 5]

In [ ]:
#Покемоны со слабой спец.атакой
df[df.Sp_attack == 10]

In [ ]:
#Покемоны со слабой спец.защитой
df[df.Sp_defence == 20]

In [ ]:
# Покемоны медленные в атаке
df[df.Speed == 5]